# 使用Pandas批量拆分与合并excel表格

In [1]:
import pandas as pd 
import os

In [2]:
os.chdir(r'H:\Coding\Python\oper_excel')

In [3]:
work_dir = r'.\course_datas\c15_excel_split_merge'
# work_dir = '.\\course_datas\\c15_excel_split_merge'
# work_dir = './course_datas/c15_excel_split_merge'

In [4]:
splits_dir = f'{work_dir}\\split'

In [5]:
splits_dir

'.\\course_datas\\c15_excel_split_merge\\split'

In [6]:
if not os.path.exists(splits_dir):
    os.mkdir(splits_dir)

In [7]:
pwd

'H:\\Coding\\Python\\oper_excel'

In [8]:
df_source = pd.read_excel(f"{work_dir}\\crazyant_blog_articles_source.xlsx")

In [9]:
df_source.head()

,id,title,tags
0,2585,Tensorflow怎样接收变长列表特征,"python,tensorflow,特征工程"
1,2583,Pandas实现数据的合并concat,"pandas,python,数据分析"
2,2574,Pandas的Index索引有什么用途？,"pandas,python,数据分析"
3,2564,机器学习常用数据集大全,"python,机器学习"
4,2561,一个数据科学家的修炼路径,数据分析


In [10]:
df_source.index

RangeIndex(start=0, stop=258, step=1)

## 将一个Excel等份拆成多个Excel
### 1. 使用df.iloc拆分一个大的dataframe
### 2. 使用dataframe.to_excel保存每个小的excel

In [17]:
# Excel拆分给如下几个人
user_names = ['lei', 'bei', 'ming', 'hong']
total_row_count = df_source.index.size
# total_row_count = df_source.shape[0]

In [19]:
split_size = total_row_count // len(user_names)
if total_row_count % len(user_names) != 0:
    split_size += 1

In [20]:
split_size

65

In [21]:
df_subs = []

In [23]:
for idx, user_name in enumerate(user_names):
    # iloc的开始索引
    begin = idx*split_size
    # iloc的结束索引
    end = begin + split_size
    df_sub = df_source.iloc[begin:end]
    df_subs.append((idx, user_name, df_sub))

In [35]:
for idx, user_name, df_sub in df_subs:
    file_name = f"{splits_dir}/crazyant_blog_articles_{idx}_{user_name}.xlsx"
    df_sub.to_excel(file_name, index = False)

## 二、合并多个excel到一个excel文件
1. 遍历文件夹，得到要合并的excel文件名列表
2. 分别读取到dataframe，给每个df添加一列用于标识来源
3. 使用pd.concat进行df批量合并
4. 将合并后的dataframe输出到excel

In [36]:
import os
excel_names = []
for excel_name in os.listdir(splits_dir):
    excel_names.append(excel_name)
excel_names

['crazyant_blog_articles_0_lei.xlsx',
 'crazyant_blog_articles_1_bei.xlsx',
 'crazyant_blog_articles_2_ming.xlsx',
 'crazyant_blog_articles_3_hong.xlsx']

In [40]:
df_list = []
for excel_name in excel_names:
    excel_path = f"{splits_dir}/{excel_name}"
    df_split = pd.read_excel(excel_path)
    username = excel_name.replace("crazyant_blog_articles_", "").replace(".xlsx", "")[2:]
    print(f'file name is {excel_name}, user name is {username}')
    df_split['username'] = username
    df_list.append(df_split)

file name is crazyant_blog_articles_0_lei.xlsx, user name is lei
file name is crazyant_blog_articles_1_bei.xlsx, user name is bei
file name is crazyant_blog_articles_2_ming.xlsx, user name is ming
file name is crazyant_blog_articles_3_hong.xlsx, user name is hong


In [42]:
df_merged = pd.concat(df_list)

In [43]:
df_merged.shape

(258, 4)

In [44]:
df_merged.head()

,id,title,tags,username
0,2585,Tensorflow怎样接收变长列表特征,"python,tensorflow,特征工程",lei
1,2583,Pandas实现数据的合并concat,"pandas,python,数据分析",lei
2,2574,Pandas的Index索引有什么用途？,"pandas,python,数据分析",lei
3,2564,机器学习常用数据集大全,"python,机器学习",lei
4,2561,一个数据科学家的修炼路径,数据分析,lei


In [45]:
df_merged['username'].value_counts()

ming    65
bei     65
lei     65
hong    63
Name: username, dtype: int64

In [48]:
df_merged.to_excel(f'{work_dir}/crazyant_blog_articles_merged.xlsx', index = False)